In [1]:
import gzip
import pickle
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, \
                                    Activation, Dropout, MaxPooling2D

def evaluate(path, model):
    X, y = pickle.load(gzip.open(path, 'rb'))
    y[y != 0] -= 2
    X = X / 255.
    acc = np.mean(model(X).numpy().argmax(axis=1) == y)
    return acc


In [2]:
path = 'flatland_train.data'
X,y = pickle.load(gzip.open(path,'rb')) 

y[y != 0] -= 2
X = X.reshape (X.shape[0],50,50,1) / 255.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

DNN modelis

In [5]:
# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=[50, 50]))
# model.add(keras.layers.Dense(64, activation="relu"))
# model.add(keras.layers.Dense(32, activation="relu"))
# model.add(keras.layers.Dense(10, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy",
#               optimizer="Adam",
#               metrics=["accuracy"])
# model.summary()

CNN modelis

In [32]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))

model.add(Dropout(0.3))

model.add(Flatten())
model.add(keras.layers.Dense(60, activation="relu"))
model.add(keras.layers.Dense(53, activation="relu"))

model.add(keras.layers.Dense(17, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_49 (Conv2D)          (None, 48, 48, 32)        320       
                                                                 
 batch_normalization_49 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_49 (MaxPoolin  (None, 24, 24, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_50 (Conv2D)          (None, 22, 22, 64)        18496     
                                                                 
 batch_normalization_50 (Bat  (None, 22, 22, 64)       256       
 chNormalization)                                                
                                                     

In [33]:
loss = model.fit(X_train, y_train, epochs=7,validation_data=[X_test,y_test])


Epoch 1/7
219/219 [==============================] - 5s 18ms/step - loss: 0.8372 - accuracy: 0.7000 - val_loss: 3.5071 - val_accuracy: 0.1520
Epoch 2/7
219/219 [==============================] - 3s 15ms/step - loss: 0.2948 - accuracy: 0.9160 - val_loss: 2.8820 - val_accuracy: 0.2380
Epoch 3/7
219/219 [==============================] - 3s 14ms/step - loss: 0.1804 - accuracy: 0.9620 - val_loss: 0.2684 - val_accuracy: 0.9240
Epoch 4/7
219/219 [==============================] - 3s 15ms/step - loss: 0.1470 - accuracy: 0.9707 - val_loss: 0.1718 - val_accuracy: 0.9637
Epoch 5/7
219/219 [==============================] - 3s 15ms/step - loss: 0.1252 - accuracy: 0.9723 - val_loss: 0.1729 - val_accuracy: 0.9700
Epoch 6/7
219/219 [==============================] - 3s 14ms/step - loss: 0.1086 - accuracy: 0.9783 - val_loss: 0.1681 - val_accuracy: 0.9770
Epoch 7/7
219/219 [==============================] - 3s 15ms/step - loss: 0.0994 - accuracy: 0.9776 - val_loss: 0.1629 - val_accuracy: 0.9707


In [35]:
model.save('model_cnn20.h5')


In [37]:
model = keras.models.load_model('model_cnn20.h5')
evaluate('flatland_train.data', model)

0.9788

In [34]:
pred = model.predict(X_test).argmax(axis=1)
print('Accuracy on test set - {0:.02%}'.format((pred == y_test).mean()))

Accuracy on test set - 97.07%
